In [51]:
import keras
from keras.datasets import mnist
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, Flatten, Embedding, regularizers
from keras.layers import Conv1D, MaxPooling1D
from keras.layers.merge import Concatenate
from keras import optimizers
from keras.preprocessing import sequence
from keras import backend as K
from w2v import train_word2vec 
from keras.utils import np_utils
import pickle, datetime
import numpy as np
import pandas as pd
from scipy.signal import convolve2d

import tensorflow as tf
from gensim import corpora
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import SnowballStemmer
from w2v import train_word2vec 
import pickle, datetime
import difflib

## Loading Data

In [26]:
df = pd.read_csv('../../Datasets/SST1_dataset/Processed_SST1.tsv', sep='\t')

raw_docs_train      = df[df.split_ind == 1]['Phrases'].values
sentiment_train     = df[df.split_ind == 1]['Label'].values
raw_docs_test       = df[df.split_ind == 2]['Phrases'].values
sentiment_test      = df[df.split_ind == 2]['Label'].values
num_labels          = len(np.unique(sentiment_train))

In [3]:
fname = '../../Datasets/SST1_dataset/sst_data'
with open(fname, 'rb') as file:
    data = pickle.load(file)
    
x_train, y_train, x_test, y_test, dictionary = data
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(9645, 37) (9645, 5) (2210, 37) (2210, 5)


In [4]:
def load_model(fname):
    model = keras.models.model_from_json(open(fname + '.json').read())
    model.load_weights(fname + '_weights.h5')
    model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=["accuracy"])
    return model

In [5]:
fname = '/home/shikhar/Datasets/Models/dodo_cnn_non_static2'
model = load_model(fname)

In [6]:
scores = model.evaluate(x_train, y_train, verbose=0)
print("Train Accuracy: %.2f%%" % (scores[1]*100))
scores = model.evaluate(x_test, y_test, verbose=0)
print("Test Accuracy: %.2f%%" % (scores[1]*100))

Train Accuracy: 96.02%
Test Accuracy: 40.95%


### Occlusion Method

In [11]:
x_test1 = x_test[0]

In [17]:
test_pred = model.predict(x_test)

In [18]:
test_pred = np.argmax(test_pred, axis=1)
y_actual = np.argmax(y_test, axis=1)

In [19]:
print(np.sum(test_pred == y_actual)*1.0 / y_actual.shape[0])

0.409502262443


In [20]:
crct_pred = (test_pred == y_actual)
ind = []
for i in range(crct_pred.shape[0]):
    if crct_pred[i] == 1:
        ind.append(i)

In [21]:
y_actual

array([2, 3, 4, ..., 4, 3, 0])

In [22]:
k = -1
print(y_actual[ind[k]])
vec = word_id_test[ind[k]]

org_pred =  model.predict(x_test[ind[k]:ind[k]+1, :])
print('Original Prediction', org_pred)

for ele in list(set(vec)):
    vec_temp = [i for i in vec if i != ele]    
    x_test1 = sequence.pad_sequences([vec_temp], maxlen=seq_len, padding="post", truncating="post")
    pred = model.predict(x_test1)
    print(dictionary.id2token[ele])
    print(pred)
#     print(dictionary.id2token[ele], '\t', pred )


0


NameError: name 'word_id_test' is not defined

In [50]:
count = 0
for layer in model.layers:
    print(count, layer.name, layer.input_shape, layer.output_shape)
    count += 1
    wts = layer.get_weights()
    for wt in wts:
        print(wt.shape)
    print('****************************************')

0 input_1 (None, 37) (None, 37)
****************************************
1 embedding (None, 37) (None, 37, 300)
(11992, 300)
****************************************
2 dropout_1 (None, 37, 300) (None, 37, 300)
****************************************
3 conv1d_1 (None, 37, 300) (None, 35, 100)
(3, 300, 100)
(100,)
****************************************
4 conv1d_2 (None, 37, 300) (None, 34, 100)
(4, 300, 100)
(100,)
****************************************
5 conv1d_3 (None, 37, 300) (None, 33, 100)
(5, 300, 100)
(100,)
****************************************
6 max_pooling1d_1 (None, 35, 100) (None, 1, 100)
****************************************
7 max_pooling1d_2 (None, 34, 100) (None, 1, 100)
****************************************
8 max_pooling1d_3 (None, 33, 100) (None, 1, 100)
****************************************
9 flatten_1 (None, 1, 100) (None, 100)
****************************************
10 flatten_2 (None, 1, 100) (None, 100)
****************************************
11 

In [74]:
max_sent_len = dictionary.seq_len

In [80]:
relu1

<tf.Tensor 'Relu_7:0' shape=(1, 35, 100) dtype=float32>

In [92]:
max_sent_len - 3 + 1

35

In [164]:
ftr_sz = [3,4,5]

inp = tf.constant(x_test[0:1,])
lbl = tf.constant(y_test[0:1,])

wts = model.get_layer('embedding').get_weights()
wEmb = tf.constant(wts[0])

emb = tf.nn.embedding_lookup(wEmb, inp)

# First convolution layer
wts = model.get_layer('conv1d_1').get_weights()
wConv1 = tf.constant(wts[0])
bConv1  = tf.constant(wts[1])

conv1 = tf.nn.conv1d(emb, wConv1, stride = 1, padding='VALID')
bias1 = conv1 + bConv1
relu1 = tf.nn.relu(bias1)

pool1 = tf.nn.max_pool([relu1], ksize = [1, 1, max_sent_len - ftr_sz[0] + 1, 1],
                       strides = [1,1,1,1], padding = 'VALID')

# Second convolution layer
wts = model.get_layer('conv1d_2').get_weights()
wConv2 = tf.constant(wts[0])
bConv2  = tf.constant(wts[1])

conv2 = tf.nn.conv1d(emb, wConv2, stride = 1, padding='VALID')
bias2 = conv2 + bConv2
relu2 = tf.nn.relu(bias2)

pool2 = tf.nn.max_pool([relu2], ksize = [1, 1, max_sent_len - ftr_sz[1] + 1, 1],
                       strides = [1,1,1,1], padding = 'VALID')

# Third convolution layer
wts = model.get_layer('conv1d_3').get_weights()
wConv3 = tf.constant(wts[0])
bConv3  = tf.constant(wts[1])

conv3 = tf.nn.conv1d(emb, wConv3, stride = 1, padding='VALID')
bias3 = conv3 + bConv3
relu3 = tf.nn.relu(bias3)

pool3 = tf.nn.max_pool([relu3], ksize = [1, 1, max_sent_len - ftr_sz[2] + 1, 1],
                       strides = [1,1,1,1], padding = 'VALID')


flat = tf.concat([pool1[0,:,0,:], pool2[0,:,0,:], pool3[0,:,0,:]], axis = 1)

wts = model.get_layer('dense_1').get_weights()
wDen1 = tf.constant(wts[0])
bDen1 = tf.constant(np.reshape(wts[1], (1, wts[1].shape[0],)))

den1 = tf.matmul(flat, wDen1)
den1f = tf.add(bDen1, den1)
den1f = tf.nn.relu(den1f)

wts = model.get_layer('dense_2').get_weights()
wDen2 = tf.constant(wts[0])
bDen1 = tf.constant(np.reshape(wts[1], (1, wts[1].shape[0],)))

den2 = tf.matmul(den1f, wDen2)
den2f = tf.add(bDen2, den2)

final = tf.nn.softmax_cross_entropy_with_logits(labels=den2f, logits=lbl)

In [168]:
final.eval(feed_dict={inp:})

<tf.Tensor 'Const_441:0' shape=(2, 5) dtype=float64>

In [442]:
embed_wt = out[1][0]
# 0 456 134 CNN rand
k = 0
list_ind = word_id_test[ind[k]]
x_input = embed_wt[list_ind, :]
y_final = y_out[:,y_actual[ind[k]]]
y_final

array([ 0.,  0.,  0.,  1.,  0.])

In [47]:
y_test[0,]
x_test[0,]

array([  551,    68,  1079,  4577,   433, 11991, 11991, 11991, 11991,
       11991, 11991, 11991, 11991, 11991, 11991, 11991, 11991, 11991,
       11991, 11991, 11991, 11991, 11991, 11991, 11991, 11991, 11991,
       11991, 11991, 11991, 11991, 11991, 11991, 11991, 11991, 11991, 11991], dtype=int32)

In [443]:
w1 = out[1][9]
y1 = w1 @ y_final
y1 = y1 * (y1 > 0)

w2 = out[1][7]
y2 = w2 @ y1
y2 = y2 * (y2 > 0) # Guided backward pass: backprop
print(y2.shape)

z1 = y2[0:50,]
z2 = y2[50:100,]
z3 = y2[100:150,]

W1 = out[1][1]
W2 = out[1][3]
W3 = out[1][5]
print(W1.shape, W2.shape, W3.shape)

x_viz = x_input


o1 = np.empty((50, ), np.float32)

i1 = np.empty((50, ), np.int32)
for i in range(50):
    output = convolve2d(x_viz, W1[:,:,i], mode='valid')
    output = output * (output > 0) 
    o1[i] = np.max(output, axis = 0)
    i1[i] = np.argmax(output, axis = 0)
    
o2 = np.empty((50, ), np.float32)
i2 = np.empty((50, ), np.int32)
for i in range(50):
    output = convolve2d(x_viz, W2[:,:,i], mode='valid')
    output = output * (output > 0) 
    o2[i] = np.max(output, axis=0)
    i2[i] = np.argmax(output, axis=0)
    
o3 = np.empty((50, ), np.float32)
i3 = np.empty((50, ), np.int32)
for i in range(50):
    output = convolve2d(x_viz, W3[:,:,i], mode='valid')
    output = output * (output > 0) 
    o3[i] = np.max(output, axis=0)
    i3[i] = np.argmax(output, axis=0)
    
print(o1.shape, o2.shape, o3.shape)

# Backward pass guided backpropogation (backprop)
t1 = o1 * z1
t2 = o2 * z2
t3 = o3 * z3

# Backward pass guided backpropogation
# t1 = z1
# t2 = z2
# t3 = z3

# t1 = o1 
# t2 = o2 
# t3 = o3 

tsum = np.sum(t1) + np.sum(t2) + np.sum(t3)

c = np.zeros((21,), np.float32)


for i in range(50):
    for j in range(3):
        c[i1[i] + j] += 1/3*t1[i]/tsum
        
for i in range(50):
    for j in range(4):
        c[i2[i] + j] += 1/4*t2[i]/tsum

for i in range(50):
    for j in range(5):
        c[i3[i] + j] += 1/5*t3[i]/tsum
        
c
np.sum(c)

(150,)
(3, 100, 50) (4, 100, 50) (5, 100, 50)
(50,) (50,) (50,)


1.0

In [444]:
raw_docs_test[ind[k]]

'If you sometimes like to go to the movies to have fun , Wasabi is a good place to start .'

In [445]:
wrds = []
sent = processed_docs_test[ind[k]]
wrd_prob = dict()

counter = 0
for i in sent:
    wrd_prob[i] = c[counter]
    counter += 1
    
import operator
# srt_list = sorted(wrd_prob.items())
sorted_x = sorted(wrd_prob.items(), key=operator.itemgetter(1), reverse=True)
sorted_x

[('wasabi', 0.23152487),
 ('good', 0.2180485),
 ('fun', 0.17998767),
 ('movi', 0.15142322),
 ('place', 0.12892881),
 ('go', 0.056666099),
 ('start', 0.020311475),
 ('like', 0.0084058866),
 ('sometim', 0.0033888198),
 ('if', 0.001314683)]

In [447]:
c

array([ 0.00131468,  0.00338882,  0.00840589,  0.0566661 ,  0.15142322,
        0.17998767,  0.23152487,  0.2180485 ,  0.12892881,  0.02031147,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ], dtype=float32)